# Importiere Pakete

In [ ]:
from pathlib import Path

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx

from IPython.display import display, HTML

plt.style.use('fivethirtyeight')
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('future.no_silent_downcasting', True)

np.random.seed(42)

# Lade Datensatz

In [ ]:
DATA_DIR = Path("./HUK_MA_CC/")

interesse = pd.read_csv(DATA_DIR/"interesse.csv", sep=",")
alter_geschlecht = pd.read_csv(DATA_DIR/"alter_geschlecht.csv", sep=",")
rest = pd.read_csv(DATA_DIR/"rest.csv", sep=";")

# Explorative Datenanalyse

## Hilfsfunktionen

In [ ]:
def create_info_table(df: pd.DataFrame) -> pd.DataFrame:
    df_merged_info = pd.DataFrame({"dtypes": df.infer_objects(False).dtypes, "num_nans": df.isna().sum(), "nunique": df.nunique()})
    df_merged_info_with_describe = pd.concat([df_merged_info, df.describe().T], axis=1)
    return df_merged_info_with_describe

In [ ]:
def print_unique_values(df: pd.DataFrame, columns: list[str]=None) -> None:
    columns = columns or df.columns
    for column in columns:
        unique_values = df[column].unique()
        nunique = len(unique_values)
        print(f"Einzigartige Elemente in Spalte '{column}': ({nunique} verschiedene Elemente)")
        print(df[column].unique(), end="\n\n")

In [ ]:
def print_unique_values2(df: pd.DataFrame, columns_to_exclude: list[str]=None, max_lines_to_print: int=None) -> None:
    columns_to_exclude = columns_to_exclude or []
    columns = [c for c in df.columns if c not in columns_to_exclude]
    max_lines_to_print = max_lines_to_print or len(df)
    for column in columns:
        unique_values = df[column].value_counts().sort_values(ascending=False).reset_index()
        unique_values.columns = ["Wert", "Anzahl"]
        unique_values["Prozent"] = (unique_values["Anzahl"] / len(df)) * 100
        unique_values_max_lines = unique_values.head(max_lines_to_print)
        print(f"Einzigartige Elemente in Spalte '{column}':\n{unique_values_max_lines}", end="\n\n")

In [ ]:
def plot_histogram(df: pd.DataFrame, name: str, bins: int, x_tick_interval: int) -> None:
    counts = df[name].value_counts().sort_index()
    counts.plot(kind='bar', color='skyblue')
    plt.xlabel(f"{name}")
    plt.ylabel("Counts")
    plt.title(f"{name}-Hist ({bins} Bins)")
    tick_positions = range(len(counts))
    plt.xticks(tick_positions[::x_tick_interval], counts.index[::x_tick_interval], rotation=45)
    plt.tight_layout(pad=1.4)
    plt.grid(None)
    plt.show()

In [ ]:
def plot_histogram2(df: pd.DataFrame, name: str, bins: int) -> None:
    df.hist(column=name, bins=bins)
    plt.xlabel(f"{name}")
    plt.ylabel("Counts")
    plt.title(f"{name}-Hist ({bins} Bins)")
    plt.tight_layout(pad=1.4)
    plt.grid(None)
    plt.show()

In [ ]:
def plot_histogram3(df: pd.DataFrame, x: str, **kwargs) -> None:
    sns.histplot(data=df, x=x, **kwargs)
    plt.xlabel(f"{x}")
    plt.ylabel("Counts")
    plt.title(f"{x}-Hist")
    plt.tight_layout(pad=1.4)
    plt.xlim(min(df[x]), max(df[x]))
    plt.grid(None)
    plt.show()

## Überblick über Tabellen

### interesse.csv

In [ ]:
interesse_info = create_info_table(interesse)
display(interesse_info)
print_unique_values2(interesse, ["id"])

### alter_geschlecht.csv

In [ ]:
alter_geschlecht_info = create_info_table(alter_geschlecht)
display(alter_geschlecht_info)
print_unique_values2(alter_geschlecht, ["id"], max_lines_to_print=100)

### rest.csv

In [ ]:
rest_info = create_info_table(rest)
display(rest_info)
print_unique_values2(rest, ["id"], max_lines_to_print=100)

## Zusammenführung der Tabellen

In [ ]:
alter_geschlecht_interesse = pd.merge(alter_geschlecht, interesse, on="id", how="inner")

In [ ]:
alter_geschlecht_interesse.describe()

**Deutlich geringere Anzahl an ID-Überschneidungen 381109 / 508146 (approx. 75%)!**

In [ ]:
df = pd.merge(alter_geschlecht_interesse, rest, on="id", how="inner").reset_index(drop=True)
df.set_index("id", inplace=True)
df.sort_index(inplace=True)
df.head()

In [ ]:
df_info = create_info_table(df)
display(df_info)
print_unique_values2(df, max_lines_to_print=100)

## Plotte Histogramme

### Interesse

In [ ]:
plot_histogram(df, "Interesse", df_info.loc["Interesse", "nunique"], 1)

### Geschlecht

In [ ]:
plot_histogram(df, "Geschlecht", df_info.loc["Geschlecht", "nunique"], 1)

### Alter

In [ ]:
plot_histogram(df, "Alter", df_info.loc["Alter", "nunique"], 5)

### Fahrerlaubnis

In [ ]:
plot_histogram(df, "Fahrerlaubnis", df_info.loc["Fahrerlaubnis", "nunique"], 1)

### Kundentreue

In [ ]:
plot_histogram(df, "Kundentreue", df_info.loc["Kundentreue", "nunique"], 20)

### Vertriebskanal

In [ ]:
plot_histogram(df, "Vertriebskanal", df_info.loc["Vertriebskanal", "nunique"], 10)

### Jahresbeitrag

In [ ]:
plot_histogram2(df, "Jahresbeitrag", 500)

In [ ]:
print(f"{int((df["Jahresbeitrag"]<=2630).sum())} Kunden zahlen höchstens 2630 € Jahresbeitrag (Peak in Histogramm)")

### Vorschaden

In [ ]:
plot_histogram(df, "Vorschaden", df_info.loc["Vorschaden", "nunique"], 1)

### Alter_Fzg

In [ ]:
plot_histogram(df, "Alter_Fzg", df_info.loc["Alter_Fzg", "nunique"], 1)

### Vorversicherung

In [ ]:
plot_histogram(df, "Vorversicherung", df_info.loc["Vorversicherung", "nunique"], 1)

### Regional_Code

In [ ]:
plot_histogram(df, "Regional_Code", df_info.loc["Regional_Code", "nunique"], 5)

## Erste Erkenntnisse zu Daten

| Merkmal        | Beschreibung  / Erkenntnisse                                                                                                                                       |
|----------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Interesse      | Binär, Zielvariable sehr unbalanciert, lediglich 12% der Kunden haben Interesse bekundet.                                                                          |
| Geschlecht     | Binär, halbwegs ausgeglichen mit einem kleinen Überschuss männlicher Kunden                                                                                        |
| Alter          | Diskret / quasi-kontinuierlich mit zwei Peaks - einem großen bei jungen Kunden und einem weiteren im mittleren Alter, sowie einem breiteren Verlauf für hohes Alter|
| Fahrerlaubnis  | Binär, Histogramm sehr einseitig (lediglich 1049 unter 508146 haben keine Fahrerlaubnis).                                                                          |
| Kundentreue    | Viele Werte (290), Histogramm sehr uniform.                                                                                                                        |
| Vertriebskanal | Konvertiere nach int, Histogramm sehr sparse (evtl. 10-20 Kanäle dominant).                                                                                        |
| Jahresbeitrag  | Hoher Peak bei 2630€ (64877 von 381109 Kunden), Schiefe Gauss-Verteilung oder Gamma-Verteilung bei höheren Jahresbeiträgen.                                        |
| Vorschaden     | Binär, quasi balanciert / nahezu gleiche Häufigkeiten.                                                                                                             |
| Alter_Fzg      | Ternär, meistens zwischen 1-2 Jahren, dann <1 Jahr, die wenigsten älter als 2 Jahre.                                                                               |
| Vorversicherung| Binär, nicht ganz balanciert; es haben mehr Kunden keine Vorversicherung.                                                                                          |
| Regional_Code  | Konvertiere nach int, Histogramm sehr sparse (evtl. 5-10 Kanäle dominant).                                                                                         |

# Feature Engineering

## Data Preprocessing

### Data-Type-Casting

Konvertiere Daten in `int64`- (diskret) bzw. `float64`- (kontinuierlich) Datentypen.

In [ ]:
df_processed = df.copy()

In [ ]:
df_processed[["Vertriebskanal", "Regional_Code", "Interesse"]] = df_processed[["Vertriebskanal", "Regional_Code", "Interesse"]].astype("int")

In [ ]:
geschlechter_dict = {"Male": 0, "Female": 1}
df_processed.loc[:, "Geschlecht"] = df_processed["Geschlecht"].replace(geschlechter_dict)
df_processed["Geschlecht"] = df_processed["Geschlecht"].astype("int")

In [ ]:
alter_dict = {'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2}
df_processed.loc[:, "Alter_Fzg"] = df_processed["Alter_Fzg"].replace(alter_dict)
df_processed["Alter_Fzg"] = df_processed["Alter_Fzg"].astype("int")

In [ ]:
vorschaden_dict = {'No': 0, 'Yes': 1}
df_processed.loc[:, "Vorschaden"] = df_processed["Vorschaden"].replace(vorschaden_dict)
df_processed["Vorschaden"] = df_processed["Vorschaden"].astype("int")

In [ ]:
df_processed.dtypes

## Binning

**Bereite Daten für eine kausale Analyse vor, indem alle Daten gebinnt werden, um sie kompatibel mit Causal-Discovery-Methoden zu machen.**

### Kategoriale bzw. ordinale Variablen

#### Alter

In [ ]:
bins_alter = [20, 30, 40, 50, 60, 100]
labels_alter = [0, 1, 2, 3, 4]
df_processed["Alter_cat"] = pd.cut(df_processed["Alter"], bins=bins_alter, labels=labels_alter, right=False)

In [ ]:
df_processed[["Alter_cat", "Alter"]].value_counts().sort_index(ascending=True)

In [ ]:
plot_histogram(df_processed, "Alter_cat", 5, 1)

#### Regional_Code

In [ ]:
top_k_reg_code = 4
top_counts_reg_code = df_processed["Regional_Code"].value_counts()
top_values_reg_code = top_counts_reg_code.nlargest(top_k_reg_code).index
top_values_reg_code

In [ ]:
df_processed["Regional_Code_cat"] = df_processed["Regional_Code"].apply(lambda x: x if x in top_values_reg_code else -1)

In [ ]:
df_processed[["Regional_Code_cat", "Regional_Code"]].value_counts().sort_values(ascending=False)

In [ ]:
plot_histogram(df_processed, "Regional_Code_cat", 5, 1)

#### Vertriebskanal

In [ ]:
top_k_kanal = 4
top_counts_kanal = df_processed["Vertriebskanal"].value_counts()
top_values_kanal = top_counts_kanal.nlargest(top_k_kanal).index
top_values_kanal

In [ ]:
df_processed["Vertriebskanal_cat"] = df_processed["Vertriebskanal"].apply(lambda x: x if x in top_values_kanal else -1)

In [ ]:
df_processed[["Vertriebskanal_cat", "Vertriebskanal"]].value_counts().sort_values(ascending=False)

In [ ]:
plot_histogram(df_processed, "Vertriebskanal_cat", 5, 1)

#### Kundentreue

In [ ]:
bins_treue = [0, 100, 200, 300]
labels_treue = [0, 1, 2]
df_processed["Kundentreue_cat"] = pd.cut(df_processed["Kundentreue"], bins=bins_treue, labels=labels_treue, right=False)

In [ ]:
df_processed[["Kundentreue_cat", "Kundentreue"]].value_counts().sort_index(ascending=True)

In [ ]:
plot_histogram(df_processed, "Kundentreue_cat", 3, 1)

### Kontinuierliche Variablen

#### Jahresbeitrag

In [ ]:
# Definiere den Schwellwert zwischen Peak (2630€) und Gauss- bzw. Gamma-Verteilung oberhalb dieses Schwellwerts
S = 2650

# Peak-Daten als eigene Kategorie markieren
from_peak = df_processed["Jahresbeitrag"] < S
df_processed["Jahresbeitrag_cat"] = np.where(from_peak, 0, 6)

# Binning für Werte oberhalb des Schwellwerts mit 4 Quantilen
from_gaussian = df_processed["Jahresbeitrag"] >= S
quantiles_beitrag, bins_beitrag = pd.qcut(df_processed.loc[from_gaussian, "Jahresbeitrag"], q=4, labels=[1, 2, 3, 4], retbins=True)
bins_beitrag_thresholds = [S] + bins_beitrag.tolist()
bins_beitrag = len(bins_beitrag_thresholds)-1
print(bins_beitrag_thresholds)
df_processed.loc[from_gaussian, "Jahresbeitrag_cat"] = quantiles_beitrag

In [ ]:
df_processed[["Jahresbeitrag", "Jahresbeitrag_cat"]].head()

In [ ]:
plot_histogram(df_processed, "Jahresbeitrag_cat", 5, 1)

# Statistische Zusammenhänge zwischen Merkmalen und Zielvariablen "Interesse"

## Hilfsfunktionen (Plotting)

In [ ]:
def plot_feature_dependencies(df, x, y, hue=None, figsize=(6.5, 6.5)):
    f, ax = plt.subplots(figsize=figsize)
    sns.despine(f, left=True, bottom=True)
    sns.scatterplot(data=df, x=x, y=y, hue=hue, sizes=(1, 6), linewidth=0, alpha=0.5, ax=ax)
    plt.tight_layout(pad=1.4)
    plt.legend(loc="best")
    plt.show()

In [ ]:
def boxplot(df, x, y, hue=None, figsize=(6.5, 6.5)):
    f, ax = plt.subplots(figsize=figsize)
    sns.boxplot(data=df, x=x, y=y, hue=hue, ax=ax, orient="h")
    sns.despine(offset=10, trim=True)
    plt.tight_layout(pad=1.4)
    plt.legend(loc="best")
    plt.show()

In [ ]:
def violinplot(df, x, y, hue=None, figsize=(6.5, 6.5)):
    f, ax = plt.subplots(figsize=figsize)
    sns.violinplot(data=df, x=x, y=y, hue=hue, split=True, inner="quart", fill=True, alpha=0.5, ax=ax, orient="h")
    sns.despine(offset=10, trim=True)
    plt.tight_layout(pad=1.4)
    plt.xlabel(f"{x}")
    plt.ylabel("Counts")
    plt.title(f"{x}-Distribution")
    plt.tight_layout(pad=1.4)
    plt.grid(None)
    plt.show()

## Conditional Probability Plots | P(Interesse|x)

### P(Interesse|Geschlecht)

In [ ]:
pd.crosstab(df_processed["Interesse"], df_processed["Geschlecht"])

In [ ]:
plot_histogram3(df=df_processed, x="Geschlecht", hue="Interesse", alpha=0.5, multiple="fill", stat="density", common_norm=True)

### P(Interesse|Vorversicherung)

In [ ]:
pd.crosstab(df_processed["Interesse"], df_processed["Vorversicherung"])

In [ ]:
plot_histogram3(df=df_processed, x="Vorversicherung", hue="Interesse", alpha=0.5, multiple="fill", stat="density", common_norm=True)

### P(Interesse|Alter_cat)

In [ ]:
pd.crosstab(df_processed["Interesse"], df_processed["Alter_cat"])

In [ ]:
plot_histogram3(df=df_processed, x="Alter_cat", hue="Interesse", bins=df_processed["Alter_cat"].nunique(), alpha=0.5, multiple="fill", stat="density", common_norm=True)

In [ ]:
plot_histogram3(df=df_processed, x="Alter", hue="Interesse", bins=df_processed["Alter"].nunique(), alpha=0.5, multiple="fill", stat="density", common_norm=True)

### P(Interesse|Alter_Fzg)

In [ ]:
pd.crosstab(df_processed["Interesse"], df_processed["Alter_Fzg"])

In [ ]:
plot_histogram3(df=df_processed, x="Alter_Fzg", hue="Interesse", alpha=0.5, multiple="fill", stat="density", common_norm=True)

### P(Interesse|Vorschaden)

In [ ]:
pd.crosstab(df_processed["Interesse"], df_processed["Vorschaden"])

In [ ]:
plot_histogram3(df=df_processed, x="Vorschaden", hue="Interesse", alpha=0.5, multiple="fill", stat="density", common_norm=True)

### P(Interesse|Fahrerlaubnis)

In [ ]:
pd.crosstab(df_processed["Interesse"], df_processed["Fahrerlaubnis"])

In [ ]:
plot_histogram3(df=df_processed, x="Fahrerlaubnis", hue="Interesse", alpha=0.5, multiple="fill", stat="density", common_norm=True)

### P(Interesse|Regional_Code_cat)

In [ ]:
pd.crosstab(df_processed["Interesse"], df_processed["Regional_Code_cat"])

In [ ]:
bins_reg_code = [-1] + sorted(top_values_reg_code.tolist())
print(bins_reg_code)
plot_histogram3(df=df_processed, x="Regional_Code_cat", hue="Interesse", bins=bins_reg_code, binrange=[min(bins_reg_code), max(bins_reg_code)], alpha=0.5, multiple="fill", stat="density",  common_norm=True)

In [ ]:
plot_histogram3(df=df_processed, x="Regional_Code", hue="Interesse", bins=df_processed["Regional_Code"].nunique(), alpha=0.5, multiple="fill", stat="density",  common_norm=True)

### P(Interesse|Kundentreue_cat)

In [ ]:
pd.crosstab(df_processed["Interesse"], df_processed["Kundentreue_cat"])

In [ ]:
plot_histogram3(df=df_processed, x="Kundentreue_cat", hue="Interesse", bins=df_processed["Kundentreue_cat"].nunique(), alpha=0.5, multiple="fill", stat="density",  common_norm=True)

In [ ]:
plot_histogram3(df=df_processed, x="Kundentreue", hue="Interesse", bins=df_processed["Kundentreue"].nunique(), alpha=0.5, multiple="fill", stat="density",  common_norm=True)

### P(Interesse|Jahresbeitrag_cat)

In [ ]:
pd.crosstab(df_processed["Interesse"], df_processed["Jahresbeitrag_cat"])

In [ ]:
plot_histogram3(df=df_processed, x="Jahresbeitrag_cat", hue="Interesse", bins=bins_beitrag, alpha=0.5, multiple="fill", stat="density",  common_norm=True)

In [ ]:
plot_histogram3(df=df_processed, x="Jahresbeitrag", hue="Interesse", bins=100, alpha=0.5, multiple="fill", stat="density", common_norm=True)

In [ ]:
violinplot(df_processed, y="Interesse", x="Jahresbeitrag", hue=None)

### P(Interesse|Vertriebskanal_cat)

In [ ]:
pd.crosstab(df_processed["Interesse"], df_processed["Vertriebskanal_cat"])

In [ ]:
bins_kanal = [-1] + sorted(top_values_kanal.tolist())
print(bins_kanal)
plot_histogram3(df=df_processed, x="Vertriebskanal_cat", hue="Interesse", bins=bins_kanal, binrange=[min(bins_kanal), max(bins_kanal)], alpha=0.5, multiple="fill", stat="density",  common_norm=True)

In [ ]:
plot_histogram3(df=df_processed, x="Vertriebskanal", hue="Interesse", bins=290, alpha=0.5, multiple="fill", stat="density", common_norm=True)

# Causal Discovery

**In diesem Abschnitt geht es darum auszuschließen, dass eines der Merkmale einen Confounder zwischen der Zielvariablen "Interesse" und anderen Merkmalen darstellt.**

**Dies ist wichtig, da man ansonsten Gefahr läuft aufgrund von Verzerrungen falsche Schlussfolgerungen zu ziehen.**

In [ ]:
from networkx.drawing.nx_pydot import to_pydot

from causallearn.search.ConstraintBased.PC import pc
from causallearn.utils.PCUtils.BackgroundKnowledge import BackgroundKnowledge
from causallearn.utils.GraphUtils import GraphUtils
from causallearn.graph.GraphNode import GraphNode

In [ ]:
df_pc = df_processed.copy()

In [ ]:
df_pc.info()

In [ ]:
df_pc = df_pc[["Geschlecht", "Alter_cat", "Regional_Code_cat", "Vertriebskanal_cat", "Kundentreue_cat", "Fahrerlaubnis", "Vorversicherung", "Alter_Fzg", "Vorschaden", "Jahresbeitrag_cat", "Interesse"]]

In [ ]:
df_pc.info()

In [ ]:
# ordinal variables
df_pc["Kundentreue_cat"] = pd.Categorical(df_pc["Kundentreue_cat"], categories=labels_treue, ordered=True)
df_pc["Alter_cat"] = pd.Categorical(df_pc["Alter_cat"], categories=labels_alter, ordered=True)
df_pc["Alter_Fzg"] = pd.Categorical(df_pc["Alter_Fzg"], categories=[0, 1, 2], ordered=True)
df_pc["Jahresbeitrag_cat"] = pd.Categorical(df_pc["Jahresbeitrag_cat"], categories=[0, 1, 2, 3, 4], ordered=True)

# categorical variables
df_pc["Regional_Code_cat"] = pd.Categorical(df_pc["Regional_Code_cat"], ordered=False)
df_pc["Vertriebskanal_cat"] = pd.Categorical(df_pc["Vertriebskanal_cat"], ordered=False)
df_pc["Geschlecht"] = pd.Categorical(df_pc["Geschlecht"], ordered=False)
df_pc["Interesse"] = pd.Categorical(df_pc["Interesse"], ordered=False)
df_pc["Fahrerlaubnis"] = pd.Categorical(df_pc["Fahrerlaubnis"], ordered=False)
df_pc["Vorversicherung"] = pd.Categorical(df_pc["Vorversicherung"], ordered=False)
df_pc["Vorschaden"] = pd.Categorical(df_pc["Vorschaden"], ordered=False)

In [ ]:
df_pc.info()

In [ ]:
df_pc.describe()

**Füge Hintergrundwissen ein, indem bestimmte kausale Abhängigkeiten zwischen Variablen von vornherein ausgeschlossen werden**

**Beispielannahme: Das `Alter` hat keinen Einfluss auf das `Geschlecht` und andersherum und sollen als unabhängige Variablen betrachtet werden.**

In [ ]:
background_knowledge = BackgroundKnowledge()
# Ausschluss von Kanten: 
# z.B. Knoten1 -> Knoten2 verboten: background_knowledge.add_forbidden_by_node(GraphNode("Knoten1"), GraphNode("Knoten2"))

# Geschlecht
background_knowledge.add_forbidden_by_node(GraphNode("Alter_cat"), GraphNode("Geschlecht"))
background_knowledge.add_forbidden_by_node(GraphNode("Vorschaden"), GraphNode("Geschlecht"))
background_knowledge.add_forbidden_by_node(GraphNode("Regional_Code_cat"), GraphNode("Geschlecht"))
background_knowledge.add_forbidden_by_node(GraphNode("Vertriebskanal_cat"), GraphNode("Geschlecht"))
background_knowledge.add_forbidden_by_node(GraphNode("Jahresbeitrag_cat"), GraphNode("Geschlecht"))
background_knowledge.add_forbidden_by_node(GraphNode("Kundentreue_cat"), GraphNode("Geschlecht"))
background_knowledge.add_forbidden_by_node(GraphNode("Fahrerlaubnis"), GraphNode("Geschlecht"))
background_knowledge.add_forbidden_by_node(GraphNode("Vorversicherung"), GraphNode("Geschlecht"))
background_knowledge.add_forbidden_by_node(GraphNode("Alter_Fzg"), GraphNode("Geschlecht"))
background_knowledge.add_forbidden_by_node(GraphNode("Interesse"), GraphNode("Geschlecht"))

# Alter_cat
background_knowledge.add_forbidden_by_node(GraphNode("Geschlecht"), GraphNode("Alter_cat"))
background_knowledge.add_forbidden_by_node(GraphNode("Vorschaden"), GraphNode("Alter_cat"))
background_knowledge.add_forbidden_by_node(GraphNode("Regional_Code_cat"), GraphNode("Alter_cat"))
background_knowledge.add_forbidden_by_node(GraphNode("Vertriebskanal_cat"), GraphNode("Alter_cat"))
background_knowledge.add_forbidden_by_node(GraphNode("Jahresbeitrag_cat"), GraphNode("Alter_cat"))
background_knowledge.add_forbidden_by_node(GraphNode("Kundentreue_cat"), GraphNode("Alter_cat"))
background_knowledge.add_forbidden_by_node(GraphNode("Fahrerlaubnis"), GraphNode("Alter_cat"))
background_knowledge.add_forbidden_by_node(GraphNode("Vorversicherung"), GraphNode("Alter_cat"))
background_knowledge.add_forbidden_by_node(GraphNode("Alter_Fzg"), GraphNode("Alter_cat"))
background_knowledge.add_forbidden_by_node(GraphNode("Interesse"), GraphNode("Alter_cat"))

# Alter_Fzg
background_knowledge.add_forbidden_by_node(GraphNode("Geschlecht"), GraphNode("Alter_Fzg"))
background_knowledge.add_forbidden_by_node(GraphNode("Vorschaden"), GraphNode("Alter_Fzg"))
background_knowledge.add_forbidden_by_node(GraphNode("Regional_Code_cat"), GraphNode("Alter_Fzg"))
background_knowledge.add_forbidden_by_node(GraphNode("Vertriebskanal_cat"), GraphNode("Alter_Fzg"))
background_knowledge.add_forbidden_by_node(GraphNode("Jahresbeitrag_cat"), GraphNode("Alter_Fzg"))
background_knowledge.add_forbidden_by_node(GraphNode("Kundentreue_cat"), GraphNode("Alter_Fzg"))
background_knowledge.add_forbidden_by_node(GraphNode("Fahrerlaubnis"), GraphNode("Alter_Fzg"))
background_knowledge.add_forbidden_by_node(GraphNode("Vorversicherung"), GraphNode("Alter_Fzg"))
background_knowledge.add_forbidden_by_node(GraphNode("Alter_cat"), GraphNode("Alter_Fzg"))

# Interesse
background_knowledge.add_forbidden_by_node(GraphNode("Interesse"), GraphNode("Alter_Fzg"))
background_knowledge.add_forbidden_by_node(GraphNode("Interesse"), GraphNode("Geschlecht"))
background_knowledge.add_forbidden_by_node(GraphNode("Interesse"), GraphNode("Vorschaden"))
background_knowledge.add_forbidden_by_node(GraphNode("Interesse"), GraphNode("Regional_Code_cat"))
background_knowledge.add_forbidden_by_node(GraphNode("Interesse"), GraphNode("Vertriebskanal_cat"))
# background_knowledge.add_forbidden_by_node(GraphNode("Interesse"), GraphNode("Jahresbeitrag_cat"))
# background_knowledge.add_forbidden_by_node(GraphNode("Interesse"), GraphNode("Kundentreue_cat"))
background_knowledge.add_forbidden_by_node(GraphNode("Interesse"), GraphNode("Fahrerlaubnis"))
# background_knowledge.add_forbidden_by_node(GraphNode("Interesse"), GraphNode("Vorversicherung"))
background_knowledge.add_forbidden_by_node(GraphNode("Interesse"), GraphNode("Alter_cat"))
background_knowledge.add_forbidden_by_node(GraphNode("Interesse"), GraphNode("Alter_Fzg"))

In [ ]:
df_pc_np = df_pc.to_numpy()

In [ ]:
column_map = {i: c for i, c in enumerate(df_pc.columns)}
column_map

In [ ]:
# chisq-Unabhängigkeitstest, da kategoriale Variablen!
estimated_pdag = pc(df_pc_np, alpha=0.05, indep_test="chisq", stable=True, background_knowledge=background_knowledge, node_names = df_pc.columns)

In [ ]:
# visualization using pydot
estimated_pdag.draw_pydot_graph()
pyd = GraphUtils.to_pydot(estimated_pdag.G)
# pyd.write_png('HUK_DAG_causal_learn.png')

**Erkenntnisse:**
1. **`Interesse` stellt tatsächlich die Zielvariable da und hat Verbindungen zu vielen Merkmalen außer `Kundentreue` und `Fahrerlaubnis` (weitere Diskussion siehe unten)**
2. **`Kundentreue` scheint für die Vorhersage des `Interesse`s keine Rolle zu spielen.**
3. **Anmerkung: Es existieren mehrere potenzielle kausale Zusammenhänge zwischen Merkmalen, die nicht ausgeschlossen werden konnten.**

In [ ]:
estimated_pdag.to_nx_graph()

In [ ]:
G = estimated_pdag.nx_graph

In [ ]:
G = nx.relabel_nodes(G, column_map)

In [ ]:
list(G.nodes)

In [ ]:
list(G.edges)

**Extrahiere die direkten (kausalen) Elternknoten von `Interesse`!**

In [ ]:
interest_parents = list(sorted(G.edges, key=lambda x: x[1]))
interest_parents = [element[0] for element in interest_parents if element[1]=="Interesse"]
interest_parents

# Modell-Vergleich und Modell-Wahl

**Probiere ein möglichst einfaches und interpretierbares Modell aus. Ich sehe 5 Möglichkeiten:**
1. Logistische Regression bzw. (Augmented) Conditional Logistic Regression (unter Annahme linearer Zusammenhänge, aber One-Hot-Encoding der kategorialen Variablen nötig)
2. Decision Tree Classifier (unter Annahme potenziell nichtlinearer Zusammenhänge, dafür allerdings kein One-Hot-Encoding der kategorialen Variablen nötig)
3. Random Forest Classifier (robuster, allerdings nicht einfach zu interpretieren, da es aus einem Ensemble von Entscheidungsbäumen besteht)
4. Gradient Boosting (sehr leistungsstark, allerdings ähnlich zum Random Forest Classifier nicht leicht zu interpretieren, da es sich auch hier um einen Ensemble-Ansatz handelt)
5. Neuronale Netze (Deep Learning) (kann hochdimensionale nichtlineare Beziehungen und versteckte Muster aufdecken, allerdings kaum interpretierbar, da BlackBox, und benötigt normalerweise sehr viele Daten).

**Für einen ersten Versuch benutze ich die einfachste und interpretierbareste Möglichkeit (1) - Logistische Regression**

**Wichtig: Egal, welches Modell herangezogen wird, nutze immer nur die direkten Elternknoten der Zielvariablen "Interesse" und keine "Mediatoren" bzw. "Collider" (siehe Kausaler Graph und Bemerkungen oben)**

# Modell-Building

## Überführe Variablen in Dummy-Variablen 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_pc.describe()

In [ ]:
df_pc.info()

In [ ]:
# Füge "Fahrerlaubnis" als Elternknoten der Zielvariable "Interesse" hinzu, obwohl sie vom PC-Algorithmus aufgrund mangelnder Ereignisse nicht erkannt wurde.
# Begründung: Die obigen Conditional Probability Plots zeigen eine Abhängigkeit der Zielvariablen "Interesse" von "Fahrerlaubnis".

df_interesse_parents = df_pc[interest_parents+["Fahrerlaubnis"]+["Interesse"]]

In [ ]:
df_encoded = pd.get_dummies(df_interesse_parents, drop_first=True)

In [ ]:
df_encoded.info()

In [ ]:
column_names = df_encoded.columns.tolist()
feature_names = column_names[:-1]
target_name = column_names[-1]

In [ ]:
feature_names

In [ ]:
target_name

In [ ]:
df_encoded.head()

## Erstelle Trainings- & Testdaten

In [ ]:
x = df_encoded[feature_names]
y = df_encoded[target_name]

In [ ]:
test_size = 0.05
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=42)

## Erstelle Logistisches Regressionsmodell

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(class_weight="balanced")

## Trainiere Logistisches Regressionsmodell

In [ ]:
model.fit(x_train, y_train)

## Mache Vorhersagen auf Testdatensatz

In [ ]:
y_pred = model.predict(x_test)

## Modellbewertung

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

In [ ]:
from sklearn.metrics import confusion_matrix

def show_confusion_heat_map(y_true, y_pred, data_label: str) -> None:
    plt.figure(figsize=(4, 3))
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues')
    plt.title(f"Konfusionsmatrix für '{data_label}'")
    plt.xlabel('Vorhersagen')
    plt.ylabel('Echte Werte')
    plt.show()

In [ ]:
from sklearn.metrics import classification_report

y_train_pred = model.predict(x_train)
show_confusion_heat_map(y_train, y_train_pred, "Trainingsdaten")
print(f"\nKlassifikations-Bericht: \n{classification_report(y_train, y_train_pred)}")

In [ ]:
y_test_pred = model.predict(x_test)
show_confusion_heat_map(y_test, y_test_pred, "Testdaten")
print(f"\nKlassifikations-Bericht: \n{classification_report(y_test, y_test_pred)}")

## Ableitung relevanter Einflussfaktoren

In [ ]:
coefficients = model.coef_[0].tolist()
intercept = float(model.intercept_[0])

print(f"\nIntercept={intercept:.2f}", end="\n\n")
print("Modell-Koeffizienten, sortiert in absteigender Stärke")
df_coefficients = pd.DataFrame({"Merkmal": feature_names, "Koeffizient": coefficients, "Odds Ratio": np.exp(coefficients)})
df_coefficients.sort_values(by="Koeffizient", ascending=False)

# Diskussion der Ergebnisse

**Diskussion der Modellergebnisse und der wichtigsten Einflussfaktoren**:

* **Positiver Beitrag**: Ein `Vorschaden` **(OR: 7.18)** erhöht die Wahrscheinlichkeit am stärksten, gefolgt von `Fahrerlaubnis` **(OR: 2.86)** und jungem `Alter_cat_1` **(OR: 1.72)**.
* **Negativer Beitrag**: Eine `Vorversicherung` **(OR: 0.02)** und bestimmte `Vertriebskanäle` senken die Wahrscheinlichkeit deutlich.
* **Moderate Effekte**: Ein hohes Fahrzeugalter `Alter_Fzg_2` **(OR: 1.24)** sowie `Regionale Faktoren` und der `Jahresbeitrag` beeinflussen das Modell nur leicht.

Die logistische Regression wurde auf gebinnten Daten trainiert, wobei eine Mischung aus ursprünglich diskreten und kontinuierlichen Merkmalen in Kategorien überführt wurde. 
Die Ergebnisse zeigen eine deutliche Diskrepanz zwischen der Modellleistung für die Klassen False (Kein Interesse) und True (Interesse) der Zielvariablen `Interesse`.

**Genauigkeit und Gesamtbewertung**: 
* Mit einer Gesamtgenauigkeit (`Accuracy`) von etwa **68 %** auf den Trainings- bzw. **69 %** auf den Testdaten scheint das Modell auf den ersten Blick eine moderate Leistung zu haben.
* Allerdings zeigt ein genauerer Blick auf die Klassifikationsmetriken, dass die Genauigkeit primär von der stark ungleichen Klassenverteilung beeinflusst wird - die Klasse False (kein Interesse) dominiert mit einem viel höheren Anteil (ca. **88 %** der Daten).

**Leistung für die Mehrheitsklasse auf Testdaten (False, kein Interesse)**: 
* Die `Precision` für False ist mit **99 %** sehr hoch, was bedeutet, dass fast alle als "False" klassifizierten Instanzen tatsächlich nicht abgeschlossen wurden.
* Der `Recall` für False liegt jedoch nur bei **65 %**, was zeigt, dass ein erheblicher Anteil der tatsächlich Uninteressierten fälschlicherweise als "True" klassifiziert wird.

**Leistung für die Minderheitsklasse auf Testdaten (True, Interesse)**:
* Die `Precision` für True ist sehr niedrig **28 %**, was bedeutet, dass viele Kunden, die als interessiert (True) klassifiziert werden, in Wirklichkeit kein Interesse haben.
* Der `Recall` für True ist dagegen extrem hoch **95 %**, was zeigt, dass fast alle tatsächlichen Interessenten korrekt erkannt werden.

**Interpretation der Ergebnisse**:
* Das Modell ist darauf optimiert, möglichst viele Interessenten zu identifizieren (hoher Recall für True), nimmt dabei aber viele False Positives in Kauf (niedrige Precision für True).
Diese Verzerrung könnte durch die binning-bedingte Informationsverringerung verstärkt worden sein. Die Diskretisierung der kontinuierlichen Variablen könnte dazu geführt haben, dass wichtige Muster in den Daten verloren gingen, die für eine präzisere Trennung der Klassen notwendig wären.

**Mögliche Verbesserungen**:
* Siehe Ausblick unten

**Fazit**:
* Das Modell erkennt potenzielle Interessenten zuverlässig (hoher Recall), produziert jedoch viele False Positives (niedrige Precision).
* Eine Verfeinerung der Features, der Schwellenwerte oder der Modellarchitektur könnte dazu beitragen, ein besseres Gleichgewicht zwischen Precision und Recall zu erreichen.

# Ausblick

Wie könnte man das Modell verbessern?

Möglichkeiten zur Verbesserung des Modells für die Vorhersage der Kundenaffinität zum Abschluss einer Kfz-Versicherung:

1. **Verwendung eines ausdrucksstärkeren Modells**: Es könnte sinnvoll sein, ein leistungsfähigeres Modell zu testen (siehe Modellvorschläge oben), da möglicherweise nichtlineare Zusammenhänge zwischen den Merkmalen bestehen, die von einem linearen Modell nicht erfasst werden.
2. **Fehler durch gebinnte Daten**: Das verwendete (lineare) Modell wurde auf gebinnten Daten trainiert, was zu einem Verlust an Detailgenauigkeit in den Daten führen kann. Die so generierten Merkmale sind weniger fein granular, wodurch insbesondere bei kontinuierlichen Merkmalen wertvolle Informationen verloren gehen. Darüber hinaus könnte das Binning selbst optimiert werden, z.B. indem andere Grenzen zwischen Kategorien gezogen werden.
3. **Untersuchung des kausalen Graphen**: Der zugrunde liegende kausale Graph (d.h. der datengenerierende Prozess) sollte genauer untersucht werden, um die Ursachen zu identifizieren, wann ein Kunde ein "Interesse" für eine Kfz-Versicherung entwickelt.
4. **Berücksichtigung unbekannter, kausaler Einflussfaktoren**: Mögliche unbekannte Einflussfaktoren im datengenerierenden Prozess bzw. im kausalen Graphen wurden bislang nicht berücksichtigt. Diese könnten jedoch eine kausale Rolle beim „Interesse“ spielen und wertvolle Hinweise darauf geben, warum ein Kunde an einer Kfz-Versicherung interessiert sein könnte.